# Unstructured Hollow sphere with DMPlex from GMsh with labels

In [1]:
import warnings
warnings.filterwarnings("ignore")
import gmsh
gmsh.initialize()
import numpy as np

In [2]:
# By default, if physical groups are defined, Gmsh will export in
# the output mesh file only those elements that belong to at least one physical
# group. To force Gmsh to save all elements, you can use
gmsh.option.setNumber("Mesh.SaveAll", 1)

In [3]:
gmsh.model.add("SphereTest")

In [4]:
ball1_tag = gmsh.model.occ.addSphere(0, 0, 0, 1.0)
ball2_tag = gmsh.model.occ.addSphere(0, 0, 0, 0.3)

In [5]:
gmsh.model.occ.cut([(3, ball1_tag)], [(3, ball2_tag)], removeObject=True, removeTool=True)

([(3, 1)], [[(3, 1)], []])

In [6]:
gmsh.option.setNumber("Mesh.CharacteristicLengthMax", 0.1)

In [7]:
gmsh.model.occ.synchronize()

In [8]:
gmsh.model.getEntities(2)

[(2, 2), (2, 3)]

In [9]:
innerSurface, outerSurface = gmsh.model.getEntities(2)

In [10]:
innerMarker, outerMarker = 1, 8
gmsh.model.addPhysicalGroup(innerSurface[0], [innerSurface[1]], innerMarker)
gmsh.model.setPhysicalName(innerSurface[1], innerMarker, "Inner Surface")
gmsh.model.addPhysicalGroup(outerSurface[0], [outerSurface[1]], outerMarker)
gmsh.model.setPhysicalName(outerSurface[1], outerMarker, "Outer Surface")

In [11]:
gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(3)

Info    : Meshing 1D...
Info    : [ 20%] Meshing curve 5 (Circle)
Info    : [ 70%] Meshing curve 8 (Circle)
Info    : Done meshing 1D (Wall 0.000259847s, CPU 0.00027s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 2 (Sphere, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 3 (Sphere, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.138118s, CPU 0.13889s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 1751 nodes...
Info    : Done tetrahedrizing 1759 nodes (Wall 0.0155243s, CPU 0.015276s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    : Done reconstructing mesh (Wall 0.0349579s, CPU 0.035544s)
Info    : Found volume 1
Info    : Found void region
Info    : It. 0 - 0 nodes created - worst tet radius 4.12016 (nodes removed 0 0)
Info    : It. 500 - 499 nodes created - worst tet radius 1.61177 (nodes removed 0 1)
Info 

In [18]:
gmsh.write("sphere_test.msh")

Info    : Writing 'sphere_test.msh'...
Info    : Done writing 'sphere_test.msh'


In [19]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.8.4
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20210428
Build host    : gmsh.info
Build options : 64Bit ALGLIB ANN Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen Fltk Gmm Hxt Jpeg[fltk] Kbipack Lapack[petsc] LinuxJoystick MathEx Med Mesh Metis Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OptHom PETSc Parser Plugins Png[fltk] Post QuadTri Solver TetGen/BR Voro++ Zlib[fltk]
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.6.0
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [20]:
gmsh.finalize()

# Import Mesh into PETSc using DMPlex

In [21]:
import sys,petsc4py
petsc4py.init(sys.argv)
from petsc4py import PETSc
import numpy as np

In [22]:
options = PETSc.Options()

In [23]:
options["dm_plex_separate_marker"] = None

In [24]:
plex = PETSc.DMPlex().createFromFile("sphere_test.msh")

In [25]:
plex.view()

DM Object: DM_0x55f7f58805a0_0 1 MPI processes
  type: plex
DM_0x55f7f58805a0_0 in 3 dimensions:
  0-cells: 4142
  1-cells: 26062
  2-cells: 42097
  3-cells: 20175
Labels:
  celltype: 4 strata with value/size (0 (4142), 6 (20175), 3 (42097), 1 (26062))
  depth: 4 strata with value/size (0 (4142), 1 (26062), 2 (42097), 3 (20175))
  Face Sets: 2 strata with value/size (1 (316), 8 (3178))


In [26]:
markers_dict = {"Inner": innerMarker,
                "Outer": outerMarker
               }

In [27]:
for key, value in markers_dict.items():
    indexSet = plex.getStratumIS("Face Sets", value)
    plex.createLabel(key)
    label = plex.getLabel(key)
    if indexSet:
        label.insertIS(indexSet, value)
    indexSet.destroy()

In [28]:
plex.view()

DM Object: DM_0x55f7f58805a0_0 1 MPI processes
  type: plex
DM_0x55f7f58805a0_0 in 3 dimensions:
  0-cells: 4142
  1-cells: 26062
  2-cells: 42097
  3-cells: 20175
Labels:
  celltype: 4 strata with value/size (0 (4142), 6 (20175), 3 (42097), 1 (26062))
  depth: 4 strata with value/size (0 (4142), 1 (26062), 2 (42097), 3 (20175))
  Face Sets: 2 strata with value/size (1 (316), 8 (3178))
  Inner: 1 strata with value/size (1 (316))
  Outer: 1 strata with value/size (8 (3178))
